In [2]:
import numpy as np

# Max Voting 

In [ ]:
from sklearn import VotingClassifier

model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('dt', model)], voting='hard')
model.fit(X_train, y_train)
model.score(X_test, y_test)

# Averaging

In [ ]:
model1 = tree.DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3 = LogisticRegression()
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

pred1 = model1.predict_proba(X_test)
pred2 = model2.predict_proba(X_test)
pred3 = model3.predict_proba(X_test)

finalpred = [pred1 + pred2 + pred3]/3

# Weighted Average

In [ ]:
model1 = tree.DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3 = LogisticRegression()
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

pred1 = model1.predict_proba(X_test)
pred2 = model2.predict_proba(X_test)
pred3 = model3.predict_proba(X_test)

finalpred = [pred1*0.3 + pred2*0.3 + pred3*0.4]/3

# Stacking

In [ ]:
def Stacking(model, train, y, test, n_fold):
    
    # Instantiate StratifiedKFold class object
    folds = StratifiedKFold(n_splits=n_fold, random_state=1)
    
    # Generate empty np array for test_preds
    test_pred = np.empty((test.shape[0], 1), float)
    
    # Generate empty np array for train_preds
    train_pred = np.empty((0, 1), float)
    
    # Iterate over the evry single split generated by StratifiedKFold
    for train_indices, val_indices in folds.split(train, y.values):
        
        # Assign training and test data according the the splits
        X_train, X_val = train.iloc[train_indices], train.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]
        
        # Train the model
        model.fit(X=X_train, y=y_train)
        # Make the predictions for test data
        train_pred = np.append(train_pred, model.predict(x_val))
        # Make predictions for test data
        test_pred = np.append(test_pred, model.predict(test))
        
    return test_pred.reshape(-1, 1), train_pred

## Create two base models

In [ ]:
# First model, DecisionTreeClassifier()
model1 = tree.DecisionTreeClassifier(random_state=1)

test_pred1, train_pred1 = Stacking(
    model=model1,
    n_fold=10,
    train=X_train,
    test=X_test,
    y=y_train,
)

train_pred1 = pd.DataFrame(train_pred1)
test_pred1 = pd.DataFrame(test_pred1)

# Second model, KNeighborsClassifier()
model2 = KNeighborsClassifier()

# Calculate results of base model2
test_pred2, train_pred2 = Stacking(
    model=model2,
    n_fold=10,
    train=X_train,
    test=X_test,
    y=y_train,
)

train_pred2 = pd.DataFrame(train_pred2)
test_pred2 = pd.DataFrame(test_pred2)

In [ ]:
# Third model is run on the predictions of the two base models

# Concatenate the train and test results from the two base
# models.
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

# Create LogisticRegression() object as meta-estimator. 
model = LogisticRegression(random_state=1)

# Fit the meta-estimator using the X and y training data 
# generated by the base estimators.
model.fit(df, y_train)

# Calculate the score by comparing the test predictions 
# generated by the base estimators with the y_test data.
model.score(df_test, y_test)

In order to simplify the above explanation, the stacking model we have created has only two levels. The decision tree and knn models are built at level zero, while a logistic regression model is built at level one. Feel free to create multiple levels in a stacking model.

 